In [ ]:
import warnings

warnings.filterwarnings('ignore')

import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.utils import shuffle

from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping

In [ ]:
def plotImagenes(imagenes, size = (15, 8)):

    fig, ax = plt.subplots(1, len(imagenes), figsize = size)
    
    for i in range(len(ax)):

        ax[i].axis('off')
        ax[i].imshow(imagenes[i], cmap = 'magma')

    plt.show()

def vgg16Model(input = (256, 256, 1), output = 6):

    kernel = (3, 3)

    modelo = tf.keras.Sequential() # Modelo secuencial.
    modelo.add(tf.keras.layers.Conv2D(64, kernel_size = kernel, padding = 'same', activation = 'relu', input_shape = input))
    modelo.add(tf.keras.layers.Conv2D(64, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2)) #

    modelo.add(tf.keras.layers.Conv2D(128, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(128, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))

    modelo.add(tf.keras.layers.Conv2D(256, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(256, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(256, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))

    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))

    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))

    modelo.add(tf.keras.layers.Flatten())

    modelo.add(tf.keras.layers.Dense(4096, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(4096, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(1000, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(256, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(output, activation = tf.keras.activations.softmax))

    return modelo

In [ ]:
PATH_CSV = '/kaggle/input/fundusdiag/fundusDiag/full_df.csv'

df = pd.read_csv(PATH_CSV)

dic = {'N' : [1, 0, 0, 0, 0],
       'D' : [0, 1, 0, 0, 0],
       'G' : [0, 0, 1, 0, 0],
       'C' : [0, 0, 0, 1, 0],
       'M' : [0, 0, 0, 0, 1]}

files = df['filename'].to_list()
labeldf = [i[2] for i in df['labels'].to_list()]

label = [dic[i] for i in labeldf if i == 'N' or i == 'D' or i == 'G' or i == 'C' or i == 'M'] # Etiquetas buenas.

normal = [i for i in range(0, len(labeldf)) if labeldf[i] == 'N']
diabetes = [i for i in range(0, len(labeldf)) if labeldf[i] == 'D']
glaucoma = [i for i in range(0, len(labeldf)) if labeldf[i] == 'G']
cataratas = [i for i in range(0, len(labeldf)) if labeldf[i] == 'C']
miopia = [i for i in range(0, len(labeldf)) if labeldf[i] == 'M']

print(len(normal), len(diabetes), len(glaucoma), len(cataratas), len(miopia))

In [ ]:
aux = '/kaggle/input/fundusdiag/fundusDiag/dataset256/'

def leerImagenes_tensor(path, files, toRead, label, MAX_IM = 1500):

    im_faltantes = MAX_IM - len(toRead)
    
    rsz_rescale = tf.keras.Sequential([tf.keras.layers.Resizing(256, 256),
                                       tf.keras.layers.Rescaling(1./255)]) # Resize y normalización de cada una de las imágenes.

    nueva = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal"), # Modelo para generar nuevas imágenes.
                                 tf.keras.layers.RandomRotation(factor = (-0.10, 0.10)),
                                 tf.keras.layers.RandomContrast(factor = (0, 1))])

    if im_faltantes <= 0:

        im = [cv2.imread(aux + files[toRead[i]]) for i in range(0, MAX_IM)]
        rsz_im = [tf.image.rgb_to_grayscale(rsz_rescale(i)).numpy() for i in im] # Reescalamiento y reescalamiento.

        print("listo!")

        return [[i, label] for i in rsz_im]

    elif im_faltantes > 0 :

        im = [cv2.imread(aux + files[toRead[i]]) for i in range(0, len(toRead))]
        rsz_im = [tf.image.rgb_to_grayscale(rsz_rescale(i)).numpy() for i in im] # Reescalamiento y reescalamiento.
        extra_im = [nueva(rsz_im[np.random.randint(0, len(toRead))]).numpy() for i in range(0, im_faltantes)] # Imágenes extra.

        total_im = np.concatenate((rsz_im, extra_im), axis = 0) # Dataset completo.

        print("listo!")

        return [[i, label] for i in total_im] # Imagen con su label.

labels = [[1, 0, 0, 0, 0],
          [0, 1, 0, 0, 0],
          [0, 0, 1, 0, 0],
          [0, 0, 0, 1, 0],
          [0, 0, 0, 0, 1]]

toread = [normal, diabetes, glaucoma, cataratas, miopia]


data = [leerImagenes_tensor(aux, files, toread[i], labels[i]) for i in range(0, len(toread))] # Imágenes y labels.

print("Shape de los datos {}".format(np.shape(data)))

div = int(len(data[0]) * 0.8) # Número de imágenes (división 80 - 20)
test = len(data[0]) - div

print("División {}".format(div))

In [ ]:
shape = (256, 256, 1)
salida = len(data)

lr = 0.1
ep = 60
n = 1
es = EarlyStopping(monitor="val_loss", patience = 20)

modelos, resultados = [], []

callbacks_list = [es]

In [ ]:
# Define tand get the number os devices. 
strategy = tf.distribute.MirroredStrategy()
print('DEVICES AVAILABLE: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
for i in range(0, n): # Número de entrenamientos.

    print("Entrenamiento: {}".format(i + 1))

    aux_train, aux_test = [i[0:div] for i in data], [i[div:len(i)] for i in data] # Separación de train y test de cada una de las clases.
    train, test = aux_train[0], aux_test[0] # Primer set de imágenes.

    for i in range(1, len(data)): # Construimos el dataset.

        train, test = np.concatenate((train, aux_train[i]), axis = 0), np.concatenate((test, aux_test[i]), axis = 0) # Concatenamos todo el dataset.

    train, test = shuffle(train), shuffle(test) # Shuffle de los datos de entrenamiento y prueba.

    img_train, img_test = np.array([train[i][0] for i in range(len(train))]), np.array([test[i][0] for i in range(len(test))]) # Imágenes de entrenamiento y testeo.
    lbl_train, lbl_test = np.array([train[i][1] for i in range(len(train))]), np.array([test[i][1] for i in range(len(test))]) # Etiquetas de entrenamiento y testeo.

    with strategy.scope():
        
        modelo = vgg16Model(output = salida) # Creamos el modelo.

    modelo.compile(optimizer = SGD(learning_rate = lr, decay = lr/ep),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

    historia = modelo.fit(img_train,
                          lbl_train,
                          epochs = ep,
                          batch_size = 64 * strategy.num_replicas_in_sync,
                          validation_data = (img_test, lbl_test),
                          callbacks = callbacks_list)

    resultados.append(historia)
    modelos.append(modelo)

In [ ]:
!mkdir '/kaggle/working/GPU_f01/'
!mkdir '/kaggle/working/GPU_f01/history/'
!mkdir '/kaggle/working/GPU_f01/model'

In [ ]:
keys = resultados[0].history.keys()

PATH = '/kaggle/working/GPU_f01/history/' # Path para guardar los resultados.

for i in range(0, len(resultados)):
    
    for j in keys: # Recorremos todas las keys de los modelos.
        
        PATH_AUX = PATH + 'modelo_1_'.format(i + 1) + j + '.txt'
        
        with open(PATH_AUX, 'w') as f:
            
            for k in resultados[i].history[j]:
                
                f.write(str(k) + '\n')

In [ ]:
PATH_MODELOS = '/kaggle/working/GPU_f01/model/'

for i in range(0, len(modelos)):
    
    modelos[i].save(PATH_MODELOS + 'modelo001_1'.format(i + 1) + '.h5')
#     conv = tf.lite.TFLiteConverter.from_saved_model(PATH_MODELOS + 'modelo01_{}'.format(i + 1))
#     tflite = conv.convert()
    
#     with open(PATH_MODELOS + 'modelo01_{}/'.format(i + 1) + 'modelo01_{}.tflite'.format(i + 1), 'wb') as f:
    
#         f.write(tflite)

In [ ]:
print("holA")